In [1]:
#Import-------------------------------------------------------------------------------
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
from urllib import request
import requests
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.service import Service
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from urllib.parse import quote

options = ChromeOptions()
options.add_argument('--headless')
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox") 
options.binary_location = '/usr/bin/chromium-browser'
driver = webdriver.Chrome(options=options)

In [2]:
driver.get('https://search.yahoo.co.jp/realtime/search?p=%23artnose&ei=UTF-8&mtype=image')

In [3]:
driver.save_screenshot('test.png')

True

In [1]:
from src import const,utils,sqlite
hashtag = '#はじめ武勇伝'
date = 14

driver = utils.get_driver(False)
url = utils.get_url(hashtag)
driver.get(url)


In [3]:
import datetime
import re

# 入力文字列
#input_string = "11分前"  # または "9月9日(土) 22:28"
def date_parse(input_string:str):
    # 現在の日時を取得
    current_time = datetime.datetime.now()

    if ("秒前" in input_string) and ("昨日" not in input_string):
        # "分前" を抽出して分数に変換
        second_ago = int(input_string.split("秒前")[0])
        
        # 指定された分数だけ過去の日時を計算
        new_time = current_time - datetime.timedelta(seconds=second_ago)

    elif ("分前" in input_string) and ("昨日" not in input_string):
        # "分前" を抽出して分数に変換
        minutes_ago = int(input_string.split("分前")[0])
        
        # 指定された分数だけ過去の日時を計算
        new_time = current_time - datetime.timedelta(minutes=minutes_ago)
    elif "昨日" in input_string:
        output_string = input_string.replace('昨日 ','')
        #　昨日の日時を設定
        time_parts = output_string.split(":")
        new_time = current_time.replace(hour=int(time_parts[0]), minute=int(time_parts[1]), second=0)
        #new_time = new_time + datetime.timedelta(days=-1)

    elif (":" in input_string) and ('月' not in input_string):
        # "16:15" のような場合、今日の日付に指定時刻を設定
        time_parts = input_string.split(":")
        new_time = current_time.replace(hour=int(time_parts[0]), minute=int(time_parts[1]), second=0)
    else:
        # "9月9日(土) 22:28" のような場合、指定されたフォーマットで日時を解析 9月9日(土) 22:28
        output_string = re.sub(r'\([^)]*\)', '', input_string)
        new_time = datetime.datetime.strptime(output_string, "%m月%d日 %H:%M")
        new_time = new_time.replace(year=current_time.year)
    return new_time

In [4]:
from bs4 import BeautifulSoup
from tqdm import tqdm
#Import-------------------------------------------------------------------------------
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
from urllib import request
import requests
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.service import Service
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from urllib.parse import quote

#-------------------------------------------------------------------------------------
# 現在の日時を取得
now = datetime.datetime.now()
date_time = datetime.datetime.now()
tweets = []
load_flag = 3
load_now = 0
while(True):

    #ツイートのリストを取得
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tweet_links = soup.find_all('div', class_='Tweet_bodyContainer__n_Cs6')

    #ツイートから各要素を取得

    for tweet in tqdm(tweet_links):
        url = ''
        try:
            url = [link for link in tweet.find_all('a') if 'pic.twitter.com' in link.text][0].text#ツイートリンク
        except:
            url = ''
        images = [i['src'] for i in [link for link in tweet.find_all('img') if 'https://rts-pctr.c.yimg.jp' in link['src']]]#画像のリスト

        like_count = 0#いいねっ
        try:
            like_count = int([a for a in tweet.find_all('a') if 'like' in a.get('href','')][0].find('span').text.replace(',',''))
        except:
            like_count = 0
        user_name = tweet.find('a','Tweet_authorID__B1U8c').text#ユーザーID
        display_name = tweet.find('span','Tweet_authorName__V3waK').text#ユーザー名
        
        date_text = tweet.find('time').text
        date_time = date_parse(date_text)

        if like_count >= 10 and len(images) > 0 and len([t for t in tweets if url == t[0]]) == 0 and url != '':
            tweets.append([
                url,
                images,
                date_time,
                user_name,
                display_name,
                like_count
            ])
    print(f'tweet count -> {len(tweets)}')
    if (now - date_time).days > date:
        print('datetime',date_time)
        print(f'date over => {(now - date_time).days}/{date}')
        break

    #一番下までスクロール
    for ind in range(8):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.25)
    sleep(3)
    try:
        load_btn = driver.find_element(By.LINK_TEXT,"もっと見る")
        load_btn.click()
    except:
        load_now = load_now + 1
        
    if load_now >= load_flag:
        print(f'button over => {load_now} / {load_flag}')
        break
    

for t in tweets:
    print(t)


100%|██████████| 40/40 [00:00<00:00, 5715.09it/s]


before date -> 18:44  || after -> 2023-09-12 18:44:00.283031
before date -> 18:44  || after -> 2023-09-12 18:44:00.284032
before date -> 20分前  || after -> 2023-09-12 20:57:35.284032
before date -> 39分前  || after -> 2023-09-12 20:38:35.284032
before date -> 41分前  || after -> 2023-09-12 20:36:35.284032
before date -> 20:10  || after -> 2023-09-12 20:10:00.285031
before date -> 20:03  || after -> 2023-09-12 20:03:00.285031
before date -> 20:03  || after -> 2023-09-12 20:03:00.285031
before date -> 19:51  || after -> 2023-09-12 19:51:00.285031
before date -> 19:32  || after -> 2023-09-12 19:32:00.285031
before date -> 19:25  || after -> 2023-09-12 19:25:00.285031
before date -> 19:14  || after -> 2023-09-12 19:14:00.286032
before date -> 19:13  || after -> 2023-09-12 19:13:00.286032
before date -> 19:08  || after -> 2023-09-12 19:08:00.286032
before date -> 19:06  || after -> 2023-09-12 19:06:00.286032
before date -> 18:36  || after -> 2023-09-12 18:36:00.286032
before date -> 18:25  || af

100%|██████████| 40/40 [00:00<00:00, 6667.68it/s]


before date -> 18:44  || after -> 2023-09-12 18:44:00.653864
before date -> 18:44  || after -> 2023-09-12 18:44:00.653864
before date -> 20分前  || after -> 2023-09-12 20:57:40.653864
before date -> 39分前  || after -> 2023-09-12 20:38:40.653864
before date -> 41分前  || after -> 2023-09-12 20:36:40.654865
before date -> 20:10  || after -> 2023-09-12 20:10:00.654865
before date -> 20:03  || after -> 2023-09-12 20:03:00.654865
before date -> 20:03  || after -> 2023-09-12 20:03:00.654865
before date -> 19:51  || after -> 2023-09-12 19:51:00.654865
before date -> 19:32  || after -> 2023-09-12 19:32:00.654865
before date -> 19:25  || after -> 2023-09-12 19:25:00.654865
before date -> 19:14  || after -> 2023-09-12 19:14:00.655864
before date -> 19:13  || after -> 2023-09-12 19:13:00.655864
before date -> 19:08  || after -> 2023-09-12 19:08:00.655864
before date -> 19:06  || after -> 2023-09-12 19:06:00.655864
before date -> 18:36  || after -> 2023-09-12 18:36:00.655864
before date -> 18:25  || af

100%|██████████| 80/80 [00:00<00:00, 5000.29it/s]


...

before date -> 18:44  || after -> 2023-09-12 18:44:00.231965
before date -> 18:44  || after -> 2023-09-12 18:44:00.231965
before date -> 22分前  || after -> 2023-09-12 20:58:30.231965
before date -> 42分前  || after -> 2023-09-12 20:38:30.231965
before date -> 44分前  || after -> 2023-09-12 20:36:30.231965
before date -> 20:10  || after -> 2023-09-12 20:10:00.231965
before date -> 20:03  || after -> 2023-09-12 20:03:00.231965
before date -> 20:03  || after -> 2023-09-12 20:03:00.231965
before date -> 19:51  || after -> 2023-09-12 19:51:00.231965
before date -> 19:32  || after -> 2023-09-12 19:32:00.231965
before date -> 19:25  || after -> 2023-09-12 19:25:00.231965
before date -> 19:14  || after -> 2023-09-12 19:14:00.231965
before date -> 19:13  || after -> 2023-09-12 19:13:00.231965
before date -> 19:08  || after -> 2023-09-12 19:08:00.231965
before date -> 19:06  || after -> 2023-09-12 19:06:00.231965
before date -> 18:36  || after -> 2023-09-12 18:36:00.231965
before date -> 18:25  || af

100%|██████████| 760/760 [00:00<00:00, 5881.56it/s]


before date -> 18:44  || after -> 2023-09-12 18:44:00.298143
before date -> 18:44  || after -> 2023-09-12 18:44:00.298143
before date -> 23分前  || after -> 2023-09-12 20:57:49.298143
before date -> 42分前  || after -> 2023-09-12 20:38:49.298143
before date -> 44分前  || after -> 2023-09-12 20:36:49.298143
before date -> 20:10  || after -> 2023-09-12 20:10:00.298143
before date -> 20:03  || after -> 2023-09-12 20:03:00.298143
before date -> 20:03  || after -> 2023-09-12 20:03:00.298143
before date -> 19:51  || after -> 2023-09-12 19:51:00.298143
before date -> 19:32  || after -> 2023-09-12 19:32:00.298143
before date -> 19:25  || after -> 2023-09-12 19:25:00.298143
before date -> 19:14  || after -> 2023-09-12 19:14:00.298143
before date -> 19:13  || after -> 2023-09-12 19:13:00.298143
before date -> 19:08  || after -> 2023-09-12 19:08:00.298143
before date -> 19:06  || after -> 2023-09-12 19:06:00.298143
before date -> 18:36  || after -> 2023-09-12 18:36:00.298143
before date -> 18:25  || af

100%|██████████| 800/800 [00:00<00:00, 6077.45it/s]


before date -> 18:44  || after -> 2023-09-12 18:44:00.435506
before date -> 18:44  || after -> 2023-09-12 18:44:00.435506
before date -> 23分前  || after -> 2023-09-12 20:58:10.435506
before date -> 42分前  || after -> 2023-09-12 20:39:10.435506
before date -> 44分前  || after -> 2023-09-12 20:37:10.435506
before date -> 20:10  || after -> 2023-09-12 20:10:00.435506
before date -> 20:03  || after -> 2023-09-12 20:03:00.435506
before date -> 20:03  || after -> 2023-09-12 20:03:00.435506
before date -> 19:51  || after -> 2023-09-12 19:51:00.435506
before date -> 19:32  || after -> 2023-09-12 19:32:00.435506
before date -> 19:25  || after -> 2023-09-12 19:25:00.435506
before date -> 19:14  || after -> 2023-09-12 19:14:00.435506
before date -> 19:13  || after -> 2023-09-12 19:13:00.435506
before date -> 19:08  || after -> 2023-09-12 19:08:00.435506
before date -> 19:06  || after -> 2023-09-12 19:06:00.435506
before date -> 18:36  || after -> 2023-09-12 18:36:00.435506
before date -> 18:25  || af

In [4]:
input_string = '9月10日(日) 17:01'
output_string = re.sub(r'\([^)]*\)', '', input_string)
print(output_string)

9月10日 17:01
